Thanks for coming to the ITHS Seminar on Communicating Your Results Visually!

# Can you predict the visual form of the figure from its caption?

In [ ]:
# in the Jupyter Notebook, use shift-enter to execute a cell
import viziometrics_chart_game as vcg

# Pick keyword(s) to focus on an area that interests you

In [ ]:
vcg.set_keywords('translational health')

# Play

In [ ]:
vcg.play()

# Fun? Let me know `@healthyalgo` or `abie@uw`